In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from keys import API_KEY
from datetime import datetime


In [3]:
def clean_text(text: str) -> str:
    text = text.strip(' \n\t')
    text = text.replace('\t','')
    text_list = text.split('\n')
    new_list = []
    for t in text_list:
        t = t.strip()
        if t != '' and not t.isspace():
            new_list.append(t)
    return '\n'.join(new_list)


In [4]:
URL = 'https://fundusze.ngo.pl/aktualne?when=ever'
r = requests.get(URL)
soup = BeautifulSoup(r.content)

In [19]:
pages = int(soup.find('div', class_='cms').text.split(' ')[1])//20+1
block_str = ('name', 'descr', 'params', 'tags', 'author')
params_str = ('obszar','nabor_do','fundusz','budzet','procent','kwota')

all_grants_list = []

for i in range(1,2): # pages+1
    psoup = BeautifulSoup(requests.get(URL+f'&page={i}').content)
    boxes = psoup.find_all('div', class_='mb5')
    grants = None
    for b in boxes:
        if b.text.strip().startswith('Znalezione konkursy'):
            grants = b
    grants = grants.find_all('div', class_='grow-1')
    for g in grants:
        gdesc = g.find('div')
        gblocks = gdesc.find_all('div', recursive=False)
        if len(gblocks) < 5:
            continue
        grant_dict = {}
        for i,name in enumerate(block_str):
            grant_dict[name] = clean_text(gblocks[i].text)
        grant_dict['tags'] = grant_dict['tags'].replace('\n','')
        gparams = gblocks[2].find_all('div',class_='lh-title')
        if len(gparams) < 6:
            print(f'not enough params for {grant_dict["name"]}')
        for i,name in enumerate(params_str):
            grant_dict[name] = clean_text(gparams[i].text).replace('\n',' ')
        print(grant_dict)
        all_grants_list.append(grant_dict)

df = pd.DataFrame(all_grants_list)
df

            

{'name': 'Otwarty konkurs ofert na realizację zadań publicznych w 2024 r. w zakresie Organizacja Młodzieżowego Forum...', 'descr': 'Otwarty konkurs ofert na realizację zadań publicznych w 2024 r. w zakresie Organizacja Młodzieżowego Forum Wolontariatu wspomagającego rozwój wspólnoty lokalnej.\nStarostwo Powiatowe w Drawsku Pomorskim, NGO - Główny specjalista ds. współpracy z organizacjami pozarządowymi i z przedsiębiorstwami ogłasza otwarty konkurs ofert na realizację zadania publicznego pod nazwą: Otwarty konkurs ofert na realizację zadań publicznych w 2024 r. w zakresie Organizacja Młodzieżowego Forum Wolontariatu wspomagającego rozwój wspólnoty lokalnej..', 'params': 'drawski\nNabór do\n29.02.2024\nPowiatowy\nŁączny budżet 15 tys. PLN\nDo 90% dofinansowania\nDotacje od 15 tys. do 15 tys.', 'tags': 'Wzmacnianie III sektora, Prawa człowieka, Działalność lokalna, Wolontariat', 'author': 'NGO - Główny specjalista ds. współpracy z organizacjami pozarządowymi i z przedsiębiorstwami', 'obs

,name,descr,params,tags,author,obszar,nabor_do,fundusz,budzet,procent,kwota
0,Otwarty konkurs ofert na realizację zadań publ...,Otwarty konkurs ofert na realizację zadań publ...,drawski\nNabór do\n29.02.2024\nPowiatowy\nŁącz...,"Wzmacnianie III sektora, Prawa człowieka, Dzia...",NGO - Główny specjalista ds. współpracy z orga...,drawski,Nabór do 29.02.2024,Powiatowy,Łączny budżet 15 tys. PLN,Do 90% dofinansowania,Dotacje od 15 tys. do 15 tys.
1,Otwarty konkurs ofert na realizację zadań publ...,Otwarty konkurs ofert na realizację zadań publ...,drawski\nNabór do\n29.02.2024\nPowiatowy\nŁącz...,Ochrona zdrowia,NGO - Główny specjalista ds. współpracy z orga...,drawski,Nabór do 29.02.2024,Powiatowy,Łączny budżet 50 tys. PLN,Do 90% dofinansowania,Dotacje od 2 tys. do 5 tys.
2,Otwarty konkurs ofert na realizację zadań publ...,Otwarty konkurs ofert na realizację zadań publ...,drawski\nNabór do\n29.02.2024\nPowiatowy\nŁącz...,Ochrona zdrowia,NGO - Główny specjalista ds. współpracy z orga...,drawski,Nabór do 29.02.2024,Powiatowy,Łączny budżet 50 tys. PLN,Do 90% dofinansowania,Dotacje od 2 tys. do 5 tys.
3,Otwarty konkurs ofert na realizację zadań publ...,Otwarty konkurs ofert na realizację zadań publ...,drawski\nNabór do\n29.02.2024\nPowiatowy\nŁącz...,"Wzmacnianie III sektora, Prawa człowieka, Wolo...",NGO - Główny specjalista ds. współpracy z orga...,drawski,Nabór do 29.02.2024,Powiatowy,Łączny budżet 27 tys. PLN,Do 90% dofinansowania,Dotacje od 2 tys. do 5 tys.
4,Otwarty konkurs ofert na realizację zadań publ...,Otwarty konkurs ofert na realizację zadań publ...,drawski\nNabór do\n29.02.2024\nPowiatowy\nŁącz...,Bezpieczeństwo,NGO - Główny specjalista ds. współpracy z orga...,drawski,Nabór do 29.02.2024,Powiatowy,Łączny budżet 15 tys. PLN,Do 90% dofinansowania,Dotacje od 2 tys. do 5 tys.
5,Otwarty konkurs ofert na realizację zadań publ...,Otwarty konkurs ofert na realizację zadań publ...,drawski\nNabór do\n29.02.2024\nPowiatowy\nŁącz...,"Ekologia, zwierzęta",NGO - Główny specjalista ds. współpracy z orga...,drawski,Nabór do 29.02.2024,Powiatowy,Łączny budżet 10 tys. PLN,Do 90% dofinansowania,Dotacje od 2 tys. do 5 tys.
6,Otwarty konkurs ofert na realizację zadań publ...,Otwarty konkurs ofert na realizację zadań publ...,drawski\nNabór do\n29.02.2024\nPowiatowy\nŁącz...,"Sport, czas wolny",NGO - Główny specjalista ds. współpracy z orga...,drawski,Nabór do 29.02.2024,Powiatowy,Łączny budżet 85 tys. PLN,Procent dofinansowania nieznany,Dotacje od 1 tys. do 5 tys.
7,Otwarty konkurs ofert na realizację zadań publ...,Otwarty konkurs ofert na realizację zadań publ...,drawski\nNabór do\n29.02.2024\nPowiatowy\nŁącz...,Kultura i tradycja,NGO - Główny specjalista ds. współpracy z orga...,drawski,Nabór do 29.02.2024,Powiatowy,Łączny budżet 40 tys. PLN,Procent dofinansowania nieznany,Dotacje od 1 tys. do 5 tys.
8,Otwarty konkurs ofert na realizację zadania z ...,Otwarty konkurs ofert na realizację zadania z ...,Gorzów Wielkopolski\nNabór do\n22.02.2024\nMie...,Polityka społeczna,Wydział Spraw Społecznych,Gorzów Wielkopolski,Nabór do 22.02.2024,Miejsko-gminny,Łączny budżet 100 tys. PLN,Procent dofinansowania nieznany,Dotacje od 1 do 100 tys.
9,Profilaktyka i rozwiązywanie problemów alkohol...,Profilaktyka i rozwiązywanie problemów alkohol...,Kuźnia Raciborska\nNabór do\n22.02.2024\nMiejs...,"Polityka społeczna, Sport, czas wolny",Referat Organizacyjny i Spraw Obywatelskich,Kuźnia Raciborska,Nabór do 22.02.2024,Miejsko-gminny,Łączny budżet 80 tys. PLN,Procent dofinansowania nieznany,Kwota dofinansowania nieznana


In [20]:
df = df.drop('params', axis=1)

In [31]:
now = datetime.now()
ts = now.strftime('%y%m%d%H%M')
df.to_csv(f"grants_{ts}_{len(df)}.csv")